In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

## Move weight initilization outside

In [2]:
mnist = input_data.read_data_sets('./MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# mb is batch_size
mb_size = 64
Z_dim = 100

X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]

# one hidden layer, add another later?
h_dim = 128

In [4]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [5]:
""" Discriminator Net model """

real_images = tf.placeholder(tf.float32, shape=[None, 784])
labels = tf.placeholder(tf.float32, shape=[None, y_dim])

# D_W1 = tf.Variable(xavier_init([X_dim + y_dim, h_dim]))
# D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

# D_W2 = tf.Variable(xavier_init([h_dim, 1]))
# D_b2 = tf.Variable(tf.zeros(shape=[1]))

# theta_D = [D_W1, D_W2, D_b1, D_b2]

In [6]:
weights = {
    "disc_H" : tf.Variable(xavier_init([X_dim + y_dim, h_dim])),
    "disc_final": tf.Variable(xavier_init([h_dim, 1])),
    "gen_H": tf.Variable(xavier_init([Z_dim + y_dim, h_dim])),
    "gen_final": tf.Variable(xavier_init([h_dim, X_dim]))
}

bias = {
    "disc_H" : tf.Variable(xavier_init([h_dim])),
    "disc_final": tf.Variable(xavier_init([1])),
    "gen_H": tf.Variable(xavier_init([h_dim])),
    "gen_final": tf.Variable(xavier_init([X_dim]))
}

In [7]:
def discriminator(x, y, reuse=None):

    inputs = tf.concat(axis=1, values=[x, y])

    # hidden layer
    D_h1 = tf.nn.relu(tf.matmul(inputs, weights["disc_H"]) + bias["disc_H"])
        
    # output layer
    D_logit = tf.matmul(D_h1, weights["disc_final"]) + bias["disc_final"]
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit

In [8]:
""" Generator Net model """
Z = tf.placeholder(tf.float32, shape=[None, Z_dim])

# G_W1 = tf.Variable(xavier_init([Z_dim + y_dim, h_dim]))
# G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

# G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
# G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

# theta_G = [G_W1, G_W2, G_b1, G_b2]

In [9]:
def generator(z, y, reuse=None):
    
    inputs = tf.concat(axis=1, values=[z, y])

    # hidden layer
    G_h1 = tf.nn.relu(tf.matmul(inputs, weights["gen_H"]) + bias["gen_H"])

    # output layer
    G_log_prob = tf.matmul(G_h1, weights["gen_final"]) + bias["gen_final"]
    # images
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob

In [10]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [11]:
def plot(samples):
    fig = plt.figure(figsize=(12, 12))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys')

    return fig

## Generator

In [12]:
G_sample = generator(Z, labels)

## Discriminator

In [13]:
# learn on real images
D_real, D_logit_real = discriminator(real_images, labels)

In [14]:
# what does reuse mean?
D_fake, D_logit_fake = discriminator(G_sample, labels)

## Discriminator Loss

### Losses Helper Function

In [15]:
def loss_func(logits_in, labels_in):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_in, labels=labels_in))

In [16]:
# D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
# D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
# D_loss = D_loss_real + D_loss_fake

In [17]:
# add smoothing factor
sm_f = 0.9
D_loss_real = tf.reduce_mean(loss_func(logits_in=D_logit_real, labels_in=tf.ones_like(D_logit_real) * sm_f))
# D_loss_real = tf.reduce_mean(loss_func(logits_in=D_logit_real, labels_in=tf.ones_like(D_logit_real)))

D_loss_fake = tf.reduce_mean(loss_func(logits_in=D_logit_fake, labels_in=tf.zeros_like(D_logit_fake)))

D_loss = D_loss_real + D_loss_fake

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## Generator Loss

In [18]:
G_loss = tf.reduce_mean(loss_func(logits_in=D_logit_fake, labels_in=tf.ones_like(D_logit_fake)))

### Optimizer

In [19]:
learning_rate = 1e-3

### Collect variables 

In [20]:
tvars = tf.trainable_variables()

d_vars = [var for var in tvars if 'dis' in var.name]
g_vars = [var for var in tvars if 'gen' in var.name]

print([v.name for v in d_vars])
print([v.name for v in g_vars])

[]
[]


In [21]:
gen_vars = [weights["gen_H"], weights["gen_final"], bias["gen_H"], bias["gen_final"]]
dis_vars = [weights["disc_H"], weights["disc_final"], bias["disc_H"], bias["disc_final"]]

In [22]:
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=dis_vars)
# D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)

In [23]:
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=gen_vars)
# G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

## Traing Session

### get next batch

In [24]:
def get_next_batch(mb_size):
    return mnist.train.next_batch(mb_size)

In [25]:
type(get_next_batch)

function

In [26]:
os.path.exists('./samples2')

True

## Reshape and Rescale images?

In [27]:
# if not os.path.exists('samples2/'):
#     os.makedir('sample2/')

In [28]:
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())
init = tf.global_variables_initializer()

In [29]:
n_sample = 16
y_sample = np.zeros(shape=[n_sample, y_dim])

for i in range(n_sample):
    c = i % 10
    y_sample[i, c] = 1
    
print(y_sample)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [30]:
epochs = 10001
saver = tf.train.Saver(var_list=gen_vars)

In [31]:
i = 0

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for it in range(200000):
        if it % 1000 == 0:
            Z_sample = sample_Z(n_sample, Z_dim)
            # y_sample = np.zeros(shape=[n_sample, y_dim])
            # generate 7 only
            # y_sample[:, 7] = 1

            samples = sess.run(G_sample, feed_dict={Z: Z_sample, labels:y_sample})

            fig = plot(samples)
            plt.savefig('samples2/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
            i += 1
            plt.close(fig)

        X_mb, y_mb = mnist.train.next_batch(mb_size)

        Z_sample = sample_Z(mb_size, Z_dim)
        _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={real_images: X_mb, Z: Z_sample, labels:y_mb})
        _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: Z_sample, labels:y_mb})

        if it % 1000 == 0:
            print('Iter: {}'.format(it))
            print('D loss: {:.4}'. format(D_loss_curr))
            print('G_loss: {:.4}'.format(G_loss_curr))
            print()
    saver.save(sess, './models/cgan_model_200000_epoch.ckpt')

Iter: 0
D loss: 1.148
G_loss: 2.7

Iter: 1000
D loss: 0.003962
G_loss: 8.854

Iter: 2000
D loss: 0.03473
G_loss: 6.393

Iter: 3000
D loss: 0.05589
G_loss: 4.724

Iter: 4000
D loss: 0.0781
G_loss: 5.605

Iter: 5000
D loss: 0.1551
G_loss: 6.439

Iter: 6000
D loss: 0.3096
G_loss: 4.31

Iter: 7000
D loss: 0.3217
G_loss: 3.955

Iter: 8000
D loss: 0.588
G_loss: 4.052

Iter: 9000
D loss: 0.5134
G_loss: 2.807

Iter: 10000
D loss: 0.8305
G_loss: 2.612

Iter: 11000
D loss: 0.4966
G_loss: 3.292

Iter: 12000
D loss: 0.3435
G_loss: 2.451

Iter: 13000
D loss: 0.6255
G_loss: 2.648

Iter: 14000
D loss: 0.7937
G_loss: 1.711

Iter: 15000
D loss: 0.7926
G_loss: 1.885

Iter: 16000
D loss: 0.6785
G_loss: 2.248

Iter: 17000
D loss: 0.7108
G_loss: 2.082

Iter: 18000
D loss: 0.6982
G_loss: 2.225

Iter: 19000
D loss: 0.7506
G_loss: 1.91

Iter: 20000
D loss: 0.7086
G_loss: 1.979

Iter: 21000
D loss: 0.7126
G_loss: 1.525

Iter: 22000
D loss: 0.8644
G_loss: 1.754

Iter: 23000
D loss: 0.7446
G_loss: 1.872

Iter: 2

In [32]:
# target = np.arange(16)
# target[-6:] = target[-6:] - 10
# sample_y = np.zeros((16, 10))
# for i, num in enumerate(target):
#     sample_y[i, num] = 1

In [48]:

n_new_samples = 16

with tf.Session() as sess:
    
    saver.restore(sess,'./models/cgan_model_200000_epoch.ckpt')
    
    for i in range(10):

        sample_z = sample_Z(n_new_samples, Z_dim)

        gen_sample = sess.run(generator(Z, labels, reuse=True),feed_dict={Z:sample_z, labels:y_sample})

        # new_samples.append(gen_sample)

        fig = plot(gen_sample)
        img_file = 'samples2/new_' + str(i) +'.png'
        plt.savefig(img_file, bbox_inches='tight')
        plt.close(fig)

INFO:tensorflow:Restoring parameters from ./models/cgan_model_200000_epoch.ckpt
